In [1]:
import pandas as pd
import requests
import json
from config import api_key1, api_key2, api_key3, api_key4, api_key5, api_key6
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress


In [2]:
#Load CSV Files with movie data
imdb_file = pd.read_csv("Resource/IMDb_All_Genres_etf_clean1.csv")
imdb_clean = imdb_file.drop(columns = ["Censor", "main_genre", "side_genre", "Runtime(Mins)"])
imdb_clean = imdb_clean.rename(columns = {"Rating": "imdb_score", "Total_Gross" : "Gross"})
imdb_clean.set_index("Movie_Title")
imdb_clean["Gross"] = imdb_clean["Gross"].replace({"Gross Unkown": ""})
imdb_clean["Gross"] = imdb_clean["Gross"].replace('[\$,]', '', regex=True)
imdb_clean["Gross"] = imdb_clean["Gross"].replace('[M,]', '', regex=True)
imdb_clean["Gross"] = pd.to_numeric(imdb_clean["Gross"])
#imdb_clean["Director"] = imdb_clean["Director"].replace('[\Directors:,]', '', regex=True)
imdb_clean.head()

,Movie_Title,Year,Director,Actors,imdb_score,Gross
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,NaN
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,534.86
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,377.85
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,292.58
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,342.55


In [3]:
#Load CSV Files with movie data
#metadata_file = pd.read_csv("Resource/movie_metadata.csv")
#metadata_clean = metadata_file.drop(columns = ["color", "num_critic_for_reviews", "duration", "director_facebook_likes", 
                             #"actor_3_facebook_likes", "actor_1_facebook_likes", "genres", "num_voted_users", 
                             #"cast_total_facebook_likes", "facenumber_in_poster", "plot_keywords", "movie_imdb_link", 
                             #"num_user_for_reviews", "language", "country", "content_rating", "aspect_ratio", "movie_facebook_likes",
                                              #"actor_2_facebook_likes"])
#metadata_clean = metadata_clean.rename(columns = {"movie_title": "Movie_Title", "title_year": "Year"})
#metadata_clean = metadata_clean.set_index("Movie_Title")
#metadata_budget = metadata_clean["budget"]
#metadata_budget.head()

In [4]:
#Load CSV Files with movie data
top1000_file = pd.read_csv("Resource/Top_1000_Highest_Grossing_Movies_Of_All_Time.csv")
top1000_file = top1000_file.drop(columns = ["Genre", "Movie Rating", "Duration", "Votes", "Logline"])
top1000_file = top1000_file.rename(columns = {"Movie Title": "Movie_Title", "Year of Realease": "Year"})
top1000_file = top1000_file.set_index("Movie_Title")
top1000_file["Year"] = top1000_file["Year"].replace({"X2016": "2016"})
top1000_file["Metascore"] = top1000_file["Metascore"].replace({"******": ""})
top1000_file["Year"] = pd.to_numeric(top1000_file["Year"])
top1000_file["Metascore"] = pd.to_numeric(top1000_file["Metascore"])
top1000_file["Gross"] = top1000_file["Gross"].replace('[\$,]', '', regex=True)
top1000_file["Gross"] = top1000_file["Gross"].replace('[M,]', '', regex=True)
top1000_file["Gross"] = top1000_file["Gross"].replace({"******": ""})
top1000_file["Gross"] = pd.to_numeric(top1000_file["Gross"])
top1000_file["Worldwide LT Gross"] = top1000_file["Worldwide LT Gross"].replace('[\$,]', '', regex=True)
top1000_file.dtypes

Year                    int64
Gross                 float64
Worldwide LT Gross     object
Metascore             float64
dtype: object

In [20]:
#Merge Dataframes into one
merge_df = pd.merge(imdb_clean, top1000_file, how = "outer", on = ["Movie_Title", "Year", "Gross"])
merge_df = merge_df.drop(columns = "Worldwide LT Gross")
merge_df.head()

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,NaN,NaN
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,534.86,84.0
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,377.85,94.0
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,292.58,74.0
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,342.55,87.0


In [6]:
merge_df.loc[merge_df["Movie_Title"] == "Avatar"]

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore
143,Avatar,2009,James Cameron,"Sam Worthington, Zoe Saldana, Sigourney Weaver...",7.8,760.51,83.0


In [27]:
merge_df["Director"].str.split(", ", expand = True, n =2)

,0,1,2
0,Rishab Shetty,None,None
1,Christopher Nolan,None,None
2,Peter Jackson,None,None
3,Christopher Nolan,None,None
4,Peter Jackson,None,None
...,...,...,...
5557,Directors:Jason Friedberg,Aaron Seltzer,None
5558,Tom Putnam,None,None
5559,Robert Iscove,None,None
5560,Bob Clark,None,None


In [23]:
# Set up additional columns
merge_df["Released"] = ""
merge_df["Rotten Tomatoes"] = ""
merge_df = merge_df.dropna(subset = ["Director"])
merge_df.count()

Movie_Title        5562
Year               5562
Director           5562
Actors             5562
imdb_score         5562
Gross              4701
Metascore           909
Released           5562
Rotten Tomatoes    5562
dtype: int64

In [28]:
response = requests.get("https://www.omdbapi.com/?t=Allegiant&apikey=e4d0015e").json()
response

{'Title': 'Allegiant',
 'Year': '2016',
 'Rated': 'PG-13',
 'Released': '18 Mar 2016',
 'Runtime': '120 min',
 'Genre': 'Action, Adventure, Mystery',
 'Director': 'Robert Schwentke',
 'Writer': 'Noah Oppenheim, Adam Cooper, Bill Collage',
 'Actors': 'Shailene Woodley, Theo James, Jeff Daniels',
 'Plot': 'After the earth-shattering revelations of Insurgent, Tris must escape with Four beyond the wall that encircles Chicago, to finally discover the shocking truth of the world around them.',
 'Language': 'English',
 'Country': 'United States',
 'Awards': '3 wins & 12 nominations',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BNjk2OTc5YzQtMjAwMS00YWY4LTk1ZWItOTgyMmRkMGU4ZmY1XkEyXkFqcGdeQXVyMzQ1MzUwMTE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.7/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '11%'},
  {'Source': 'Metacritic', 'Value': '33/100'}],
 'Metascore': '33',
 'imdbRating': '5.7',
 'imdbVotes': '131,911',
 'imdbID': 'tt3410834',
 'Type': 'mo

In [42]:
response["Metascore"]

'33'

In [43]:
#Iterate through merged_df Dataframe
from time import sleep
counter = 0
baseurl = "https://www.omdbapi.com/?t="
print("Accessing API")

for index, row in merge_df.iterrows():
    counter += 1
    if counter < 950:
        apikey = api_key1
    elif counter < 1900:
        apikey = api_key2
    elif counter < 2850:
        apikey = api_key3
    elif counter < 3800:
        apikey = api_key4
    elif counter < 4750:
        apikey = api_key5
    elif counter < 5562:
        apikey = api_key6
    
    #Get movie title from merge_df
    movie_title = merge_df.loc[index, "Movie_Title"]
    
    #API Request
    time.sleep(0.01)
    filminfo = requests.get(f"{baseurl}{movietitle}&apikey={apikey}").json()
    
    #Store Release Date, Rotten Tomatoes, Metascore,Box Office
    merge_df.loc[index, "Released"] = filminfo["Released"]
    merge_df.loc[index, "Rotten Tomatoes"] = filminfo["Ratings"][1]["Value"]
    merge_df.loc[index, "Metascore"] = filminfo["Metascore"]
    merge_df.loc[index, "Gross"] = filminfo["BoxOffice"]
    
#Display Data
merge_df

Accessing API


ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer'))

In [46]:
merge_df

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore,Released,Rotten Tomatoes
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,"$785,221,649",83,18 Dec 2009,81%
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,"$785,221,649",83,18 Dec 2009,81%
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,"$785,221,649",83,18 Dec 2009,81%
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,"$785,221,649",83,18 Dec 2009,81%
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,"$785,221,649",83,18 Dec 2009,81%
...,...,...,...,...,...,...,...,...,...
5557,Disaster Movie,2008,"Directors:Jason Friedberg, Aaron Seltzer","Carmen Electra, Vanessa Lachey, Nicole Parker,...",1.9,14.19,NaN,,
5558,The Hottie & the Nottie,2008,Tom Putnam,"Paris Hilton, Joel David Moore, Christine Laki...",1.9,0.03,NaN,,
5559,From Justin to Kelly,2003,Robert Iscove,"Kelly Clarkson, Justin Guarini, Katherine Bail...",1.9,4.92,NaN,,
5560,Superbabies: Baby Geniuses 2,2004,Bob Clark,"Jon Voight, Scott Baio, Vanessa Angel, Skyler ...",1.5,9.11,NaN,,


In [64]:
merge_df.iloc[512,7]

''

In [81]:
directorframe = merge_df["Director"].str.split(", ", expand = True, n = 1)
directorframe= directorframe.rename(columns = {directorframe.columns[0]: "Director"})
directorframe
merge_df["Director"] = directorframe["Director"]
merge_df

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore,Released,Rotten Tomatoes
0,Kantara,2022,Rishab Shetty,"Rishab Shetty, Sapthami Gowda, Kishore Kumar G...",9.3,"$785,221,649",83,18 Dec 2009,81%
1,The Dark Knight,2008,Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",9.0,"$785,221,649",83,18 Dec 2009,81%
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,"Elijah Wood, Viggo Mortensen, Ian McKellen, Or...",9.0,"$785,221,649",83,18 Dec 2009,81%
3,Inception,2010,Christopher Nolan,"Leonardo DiCaprio, Joseph Gordon-Levitt, Ellio...",8.8,"$785,221,649",83,18 Dec 2009,81%
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,"Elijah Wood, Ian McKellen, Viggo Mortensen, Or...",8.8,"$785,221,649",83,18 Dec 2009,81%
...,...,...,...,...,...,...,...,...,...
5557,Disaster Movie,2008,Directors:Jason Friedberg,"Carmen Electra, Vanessa Lachey, Nicole Parker,...",1.9,14.19,NaN,,
5558,The Hottie & the Nottie,2008,Tom Putnam,"Paris Hilton, Joel David Moore, Christine Laki...",1.9,0.03,NaN,,
5559,From Justin to Kelly,2003,Robert Iscove,"Kelly Clarkson, Justin Guarini, Katherine Bail...",1.9,4.92,NaN,,
5560,Superbabies: Baby Geniuses 2,2004,Bob Clark,"Jon Voight, Scott Baio, Vanessa Angel, Skyler ...",1.5,9.11,NaN,,


In [82]:
actorframe = merge_df["Actors"].str.split(", ", expand = True, n = 1)
actorframe= actorframe.rename(columns = {actorframe.columns[0]: "Actors"})
actorframe
merge_df["Actors"] = actorframe["Actors"]
merge_df

,Movie_Title,Year,Director,Actors,imdb_score,Gross,Metascore,Released,Rotten Tomatoes
0,Kantara,2022,Rishab Shetty,Rishab Shetty,9.3,"$785,221,649",83,18 Dec 2009,81%
1,The Dark Knight,2008,Christopher Nolan,Christian Bale,9.0,"$785,221,649",83,18 Dec 2009,81%
2,The Lord of the Rings: The Return of the King,2003,Peter Jackson,Elijah Wood,9.0,"$785,221,649",83,18 Dec 2009,81%
3,Inception,2010,Christopher Nolan,Leonardo DiCaprio,8.8,"$785,221,649",83,18 Dec 2009,81%
4,The Lord of the Rings: The Two Towers,2002,Peter Jackson,Elijah Wood,8.8,"$785,221,649",83,18 Dec 2009,81%
...,...,...,...,...,...,...,...,...,...
5557,Disaster Movie,2008,Directors:Jason Friedberg,Carmen Electra,1.9,14.19,NaN,,
5558,The Hottie & the Nottie,2008,Tom Putnam,Paris Hilton,1.9,0.03,NaN,,
5559,From Justin to Kelly,2003,Robert Iscove,Kelly Clarkson,1.9,4.92,NaN,,
5560,Superbabies: Baby Geniuses 2,2004,Bob Clark,Jon Voight,1.5,9.11,NaN,,
